In [ ]:
!pip install tensorflow

In [15]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [18]:
df = pd.read_csv(r'C:\Users\Legend\Downloads\train.csv')
df.head()

,text,label
0,"rt @user olha quem chegouuuuu, nossos queridin...",0
1,veio umas teorias muito loucas na minha cabeça...,1
2,@user @user 😂😂😂😂mais nao tinha falado ontem qu...,0
3,rt @user quer ser filha da puta logo comigo qu...,1
4,vai besta 😂😂😂😂 casquei com a ultima foto,1


In [19]:
# prompt: make a regex that removes all non alphabet characters and numbers and makes them lower case as well and then rename the dataset with the same name

import re

# Remove all non-alphabet characters and numbers, and convert to lowercase
df['text'] = df['text'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]', ' ', x).lower())

# Rename the dataset with the same name
df = df.rename(columns={'text': 'text'})

In [20]:
# prompt: tokensize the text and then add padding of the max length

# Tokenize the text column
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df['text'])

# Convert the text to sequences of tokens
sequences = tokenizer.texts_to_sequences(df['text'])

# Pad the sequences to the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences
maxlen = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=maxlen)


In [21]:
padded_sequences.shape

(16800, 66)

In [22]:
# prompt: make an embedding layer at the start and then an BiDirectional LSTM layer and then finnaly a linear layer for binary classification 

# Create the model
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(10000,66),  # Embedding layer
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),  # BiDirectional LSTM layer
  tf.keras.layers.Dense(1, activation='sigmoid')  # Linear layer for binary classification
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(padded_sequences, df['label'], epochs=5, validation_split=0.2)

Epoch 1/5
420/420 ━━━━━━━━━━━━━━━━━━━━ 11s 20ms/step - accuracy: 0.6339 - loss: 0.6331 - val_accuracy: 0.7315 - val_loss: 0.5288
Epoch 2/5
420/420 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.8011 - loss: 0.4291 - val_accuracy: 0.7473 - val_loss: 0.5163
Epoch 3/5
420/420 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.8480 - loss: 0.3486 - val_accuracy: 0.7280 - val_loss: 0.5522
Epoch 4/5
420/420 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.8767 - loss: 0.2858 - val_accuracy: 0.7247 - val_loss: 0.6154
Epoch 5/5
420/420 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.9029 - loss: 0.2329 - val_accuracy: 0.7155 - val_loss: 0.7461


In [24]:
# prompt: evaluate the model and test it

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(padded_sequences, df['label'])
print('Test Accuracy:', test_acc)

# Test the model on a new sentence
new_sentence = "I love this movie!"
new_sequence = tokenizer.texts_to_sequences([new_sentence])
padded_new_sequence = pad_sequences(new_sequence, maxlen=maxlen)
prediction = model.predict(padded_new_sequence)
print('Prediction:', prediction)

525/525 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9187 - loss: 0.1977
Test Accuracy: 0.8808333277702332
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Prediction: [[0.00071525]]


In [26]:
# Test the model on a new sentence
new_sentence = "user user mais nao tinha falado ontem que n o ia ser patrocinado nada que pudesse violar a moralidade da fam lia brasileira"
new_sequence = tokenizer.texts_to_sequences([new_sentence])
padded_new_sequence = pad_sequences(new_sequence, maxlen=maxlen)
prediction = model.predict(padded_new_sequence)
# Convert the probability prediction to binary label
binary_prediction = 1 if prediction >= 0.5 else 0

# Print the binary prediction
print('Binary Prediction:', binary_prediction)
print('Prediction:', prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Binary Prediction: 0
Prediction: [[0.00065121]]


In [29]:
test_df = pd.read_csv(r"C:\Users\Legend\Downloads\test.csv")
test_df.head()

,id,text
0,0,@user nossa mano te odeio na moral kkkkkkkkkkk...
1,1,@user @user a edição ficou muito boa! kkkkkkkk...
2,2,"@user largada vá, visto que a vaca anda à solta 😂"
3,3,"poxa, eu queria ganhar um boné. alguém me da u..."
4,4,"@user amiga, tudo bem, não precisa se desculpa..."


In [ ]:
# prompt: loop over the test_df and then predict each of the values in the dataset and then add a column in the test_df named label and store the result of the prediction for that sentence

# Create a new column in the test_df to store the predictions
test_df['label'] = np.nan

# Iterate over the test_df and predict each sentence
for index, row in test_df.iterrows():
  # Tokenize the sentence
  new_sequence = tokenizer.texts_to_sequences([row['text']])
  # Pad the sequence to the same length
  padded_new_sequence = pad_sequences(new_sequence, maxlen=maxlen)
  # Predict the sentiment
  prediction = model.predict(padded_new_sequence)
  # Convert the probability prediction to binary label
  binary_prediction = 1 if prediction >= 0.5 else 0
  # Store the prediction in the 'label' column
  test_df.at[index, 'label'] = binary_prediction

# Print the updated test_df
test_df.head()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━